In [10]:
from visanalysis.analysis import imaging_data, shared_analysis
from visanalysis.util import plot_tools
import matplotlib.pyplot as plt
import numpy as np
import os
from two_photon_analysis import medulla_analysis as ma

In [11]:
# %% Analysis Parameters

fileOne = '/Users/averykrieger/Documents/local_data_repo/20220526'
fileOneName = '2022-05-26'
fileTwo = '/Users/averykrieger/Documents/local_data_repo/20220527'
fileTwoName = '2022-05-27'
fileThree = '/Volumes/ROG2TBAK/data/bruker/20220718'
fileThreeName = '2022-07-18' #Alt = 5, 9, 14, 18

roi_sets = ['distal_rois-standard', 'medial_1_rois-standard','medial_2_rois-standard','proximal_rois-standard']

In [12]:
for i in range(len(roi_sets)):

    file_directory = fileThree
    file_name = fileThreeName
    alt_pre_time = 1
    save_path = '/Users/averykrieger/Documents/local_data_repo/figs/'
    series_number = 5
    roi_name = roi_sets[i]
    opto_condition = True
    vis_stim_type = 'single' # 'spatiotemporal' or 'single'
    displayFix = True
    saveFig = True
    dff = True
    # Handling the DF/F or DF or Raw options for the Ca traces
    if dff == False:
        df = True
    else:
        df = False

    # Load up that data
    ID, roi_data = ma.dataLoader(file_directory = file_directory,
                          save_path = save_path,
                          file_name = file_name,
                          series_number = series_number,
                          roi_name = roi_name,
                          opto_condition = opto_condition,
                          displayFix = displayFix,
                          saveFig = saveFig)
    # saveNames for figures
    if df == True:
        saveName = save_path+str(file_name)+' | SeriesNumber'+' | DF is '+str(df)+' | '+str(series_number)+' | '+str(roi_name) + '.pdf'
    else:
        saveName = save_path+str(file_name)+' | SeriesNumber'+' | DFF is '+str(dff)+' | '+str(series_number)+' | '+str(roi_name) + ' | alt pre time: ' + str(alt_pre_time)+ '.pdf'

    # Find out if the stimuli Combos were good
    if vis_stim_type == 'spatiotemporal':
        ma.stimComboChecker(ID)

    #ID.getStimulusTiming(plot_trace_flag=True)

    # Calculate Response Metrics so that they can be plotted
    noptoMaxes, noptoMeans = ma.getResponseMetrics(ID = ID, roi_data = roi_data,
                                                  dff = dff, df = df,
                                                  vis_stim_type = vis_stim_type,
                                                  alt_pre_time = alt_pre_time,
                                                  opto_condition = False,
                                                  silent = False)

    print('\n\nFinished noptoMaxes, noptoMeans!\n')
    yoptoMaxes, yoptoMeans = ma.getResponseMetrics(ID = ID, roi_data = roi_data,
                                                  dff = dff, df = df,
                                                  vis_stim_type = vis_stim_type,
                                                  alt_pre_time = alt_pre_time,
                                                  opto_condition = True,
                                                  silent = False)
    print('\n\nFinished yoptoMaxes, yoptoMeans!\n')


    # PLOT traces of each ROI
    ma.plotConditionedROIResponses(ID = ID, roi_data = roi_data,
                                   opto_condition = opto_condition,
                                   vis_stim_type = vis_stim_type,
                                   alt_pre_time = alt_pre_time,
                                   dff=dff, df = df, save_path = save_path,
                                   saveFig=saveFig, saveName = saveName)

AssertionError: No hdf5 file found at 
 /Volumes/ROG2TBAK/data/bruker/20220718/2022-07-18.hdf5, check your filepath